In [ ]:
import pandas as pd
import numpy as np

#### Einlesen des Datensatzes

In [ ]:
X = pd.read_csv('diabetes.csv')
X.head()

#### Abspalten der Labels

In [ ]:
y = X.iloc[:,-1]
X = X.drop(X.columns[-1], axis=1)

#### Verteilung der Labels

In [ ]:
y.value_counts(normalize=True)

# Aufgabe 1: Modellbewertung mittels Kreuzvalidierung

#### a) Einfache Holdout-Methode

Wenden Sie die einfache Holdout-Methode an, um die Performance eines DecisionTrees-Classifiers mit den Parametern *max_depth*=6 und *min_samples_leaf*=3 zu bewerten. Trainieren Sie diesen mit sklearn auf einem Trainingsdatensatz, der 90% der Daten enthält, und werten Sie anschließend die Accuracy (=Anteil aller Instanzen, die korrekt klassifiziert wurden) auf dem Trainings- und auf dem Testdatensatz aus. Welche Beobachtung machen Sie? 
<br><br>
Hinweis: Zur Berechnung der Accuracy können Sie die Methode **score** der Klasse DecisionTreeClassifier verwenden.

In [ ]:
#TODO

#### b) $k$-fache Kreuzvalidierung anwenden

Wiederholen Sie nun Teilaufgabe a), indem Sie $10$-fache Kreuzvalidierung durchführen. Verwenden Sie dazu die Klasse <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html">*StratifiedKFold*</a>. Geben Sie die mittlere gemessene Accuracy (gemittelt über die 10 Testdatensätze) aus. Geben Sie außerdem für jeden einzelnen der 10 Durchläufe die Accuracy auf dem jeweiligen Trainings- und Testdatensatz an. Vergleichen Sie mit dem Ergebnis aus Teilaufgabe a). Bewerten Sie die Varianz des Modells.

In [ ]:
#TODO

# Aufgabe 2: Hyperparameteroptimierung durch GridSearch

#### a) $k$-fache Kreuzvalidierung

Erläutern Sie das Prinzip der $k$-fachen Kreuzvalidierung zur Hyperparameter-Optimierung. Geben Sie genau an, auf welchem Datensatz die einzelnen Trainings- und Auswertungsschritte jeweils vorgenommen werden.

#### b) Train-Test-Split

Splitten Sie den Datensatz in einen Trainings- und einen Testdatensatz auf, wobei der Testdatensatz 20% der Samples enhalten soll. 

In [ ]:
#TODO

#### c) GridSearchCV

Wenden Sie $10$-fache Kreuzvalidierung auf dem Trainingsdatensatz an, um die besten Werte für die Hyperparameter *max_depth* und *min_samples_leaf* durch Rastersuche (engl. "grid search") zu ermitteln. Testen Sie für *max_depth* die Werte $\{1,2,3,4,5,6,7,8,9,10,15,20\}$ und für 
*min_samples_leaf* die Werte $\{1,5,10,20\}$. Geben Sie für jede mögliche Kombination die durch $k$-fache Kreuzvalidierung erhaltene mittlere Accuracy aus. Für welche Werte-Kombination ist diese am größten?

Hinweise: <br>
- Klasse <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html">GridSearchCV</a> aus dem Paket sklearn.model_selection
- Die Attribute *best_params_* und  *best_score_* enthalten die optimalen Parameter und die zugehörige Accuracy
- Das Attribut *cv_results_* enthält die Infos zu allen Parameter-Kombinationen

In [ ]:
#TODO

#### d) Modelltraining mit den optimalen Parametern

Trainieren Sie nun einen DecisionTreeClassifier mit den optimalen Parametern auf dem Trainingsdatensatz und werten Sie die Accuracy auf dem Trainings- und auf dem Testdatensatz aus. Was beobachten Sie? Vergleichen Sie mit den Ergebnissen aus Teilaufgabe c)

In [ ]:
#TODO

#### e) Modellbewertung: Verzerrung und Varianz

Untersuchen/Bewerten Sie die Verzerrung und die Varianz des resultierenden Modells.

In [ ]:
#TODO (hier muss ggf. kein neuer Code geschrieben werden)

# Aufgabe 3: Regularisierung bei der polynomialen Regression

#### Erzeugung von Zufallsdaten

In [ ]:
X = np.array([0, 1, 2, 2.5, 3, 3.6, 4, 4.7, 5])
y = 0.5*X + 1 + 0.3*np.random.randn(len(X))

X = X/np.max(X)
y = y/np.max(y)

#### Visualisierung der Datenpunkte

In [ ]:
import matplotlib.pyplot as plt
plt.plot(X,y,'.')

#### a) Polynomiale Regression: Parameterbestimmung durch Aufstellen und Lösen der Normalgleichungen

Implementieren Sie eine Funktion namens **polyreg**, die die Parameter $\vec{\theta}=(\theta_0,\theta_1,\ldots,\theta_d)$ einer polynomialen Regression mit Hilfe eines Least-Squares-Ansatzes berechnet. Es soll also in Abhängigkeit vom Polynomgrad $d$ eine Modellfunktion der Form $f_{\vec{\theta}}(x)=\theta_0+\theta_1x+\theta_2x^2+\ldots+\theta_dx^d$ bestimmt werden. Stellen Sie dazu die Normalgleichungen auf (s. Vorlesung über Lineare Regression) und lösen Sie diese.

In [ ]:
def polyreg(X, y, d, L):
    """
    calculates the coefficients of a polynomial regression function 
    
    Parameters:
        X (array): training examples for the input variable (1-dimensional array)
        y (array): output values corresponding to X (1-dimensional array)
        d (int): degree of the polynomial to be fit
        
    Returns:
        theta (array): coefficients of the regression function
    """
    
    #TODO
    
    return theta



#### b) Modellanwendung

Fitten Sie mit Hilfe der in a) implementierten Funktion ein Polynom 10. Grades an die gegebenen Daten. Zeichnen Sie das Polynom mit in die Grafik ein. Welche Beobachtung machen Sie?
<br><br>
Hinweis: mit der Funktion **numpy.polyval** kann man ein Polynom mit gegebenen Koeffizienten auswerten. Dazu muss ggf. die Reihenfolge des Parametervektors theta umgekehrt werden. Dies leistet die Funktion **numpy.flipud**

In [ ]:
#TODO

#### c) Regularisierung

Erweitern Sie die in a) geschriebene Funktion, indem Sie Regularisierung "einbauen". Der Regularisierungsparameter L soll als zusätzliches Argument übergeben werden. Überlegen Sie sich zunächst, wie die Normalgleichungen für die regularisierte polynomiale Regression lauten. Testen Sie anschließend für L=0 und L=1000 die Funktion. Visualisieren Sie jeweils die resultierende Modellfunktion.

In [ ]:
#TODO